In [46]:
# Data manipulation
import pandas as pd
import numpy as np
# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 18
%matplotlib inline

MAX_EVALS = 5
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer



DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [47]:
import json
import shlex
import subprocess
import logging
from pprint import pprint
import re
import os


In [48]:
def runthebenchmark(hyperparameters):
    os.chdir('/home/meghaagr/project/progress/active/../')
    storeinfile(hyperparameters)
    out=subprocess.Popen(["python3","read_config.py"], shell=False, stdout=subprocess.PIPE)
    logging.basicConfig(level=logging.DEBUG)
    output=out.stdout.read().decode('utf-8')
    print(output)
    value = 312 #output.split(" ")[5]
    value = float(value)
    print(value)
    return float(1/value)
    #logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [54]:
def storeinfile(hyperparameters):
    data={"mpi": {"romio_ds_read": "enable", "romio_ds_write": "disable"}, "lfs": {"setstripe": {"size": "16777216", "count": 4}}}
    data["lfs"]["setstripe"]["size"] = int(hyperparameters["setstripe-size"])
    data["lfs"]["setstripe"]["count"] = int(hyperparameters["setstripe-count"])

    data["mpi"]["romio_ds_read"] = hyperparameters["romio_ds_read"]
    data["mpi"]["romio_ds_write"] = hyperparameters["romio_ds_write"]
    data["mpi"]["romio_cb_read"] = hyperparameters["romio_cb_read"]
    data["mpi"]["romio_cb_write"] = hyperparameters["romio_cb_write"]
    data["mpi"]["cb_buffer_size"] = str(int(hyperparameters["cb_buffer_size"]))
    with open("confex.json","w") as fp:
        json.dump(data,fp)
    print(data)


In [55]:
def objective(hyperparameters):
    global ITERATION
    ITERATION += 1
    start = timer()
    result = runthebenchmark(hyperparameters)
    run_time = timer() - start
    print({'result': result, 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK})
    return {'loss': float(result), 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK}
    

In [63]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

space = {
    'romio_ds_read' : hp.choice('romio_ds_read',['enable','disable']),
    'romio_ds_write' : hp.choice('romio_ds_write',['enable','disable']),
    'romio_cb_read' : hp.choice('romio_cb_read',['enable','disable']),
    'romio_cb_write' : hp.choice('romio_cb_write',['enable','disable']),
    'cb_buffer_size' : 1048576*hp.qnormal('cb_buffer_size',512,128,4),
    'setstripe-size' : 65536*(hp.qloguniform('setstripe-size',0,3,1)),
    'setstripe-count' : hp.quniform('setstripe-count',0,20,1)
    
}

In [73]:
x = sample(space)
print(x)

{'cb_buffer_size': 750780416.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'romio_ds_read': 'disable', 'romio_ds_write': 'disable', 'setstripe-count': 13.0, 'setstripe-size': 327680.0}


In [74]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest


from hyperopt import Trials

# Record results
trials = Trials()

from hyperopt import fmin


ITERATION = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials, max_evals = MAX_EVALS)


  0%|          | 0/5 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.009360 seconds
INFO:hyperopt.tpe:TPE using 0 trials


{'mpi': {'romio_ds_read': 'disable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'cb_buffer_size': '411041792'}, 'lfs': {'setstripe': {'size': 65536, 'count': 4}}}
S3D-IO 50-50-100-2-2-4-1 513.51 0.03 0.06 221.80 0.15 0.69 0.05 0.01 65536 4 disable enable disable disable 411041792 

312.0                                              
{'result': 0.003205128205128205, 'hyperparameters': {'cb_buffer_size': 411041792.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'romio_ds_read': 'disable', 'romio_ds_write': 'enable', 'setstripe-count': 4.0, 'setstripe-size': 65536.0}, 'iteration': 1, 'iteration_time': 5.071485591121018, 'status': 'ok'}
 20%|██        | 1/5 [00:05<00:20,  5.10s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.017930 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'cb_buffer_size': '427819008'}, 'lfs': {'setstripe': {'size': 131072, 'count': 7}}}
S3D-IO 50-50-100-2-2-4-1 240.32 0.03 0.13 54.82 0.15 2.78 0.06 0.01 131072 7 enable enable disable disable 427819008 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'cb_buffer_size': 427819008.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 7.0, 'setstripe-size': 131072.0}, 'iteration': 2, 'iteration_time': 6.95705162268132, 'status': 'ok'}
 40%|████      | 2/5 [00:12<00:17,  5.67s/it, best loss: 0.003205128205128205]

...

{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'cb_buffer_size': '536870912'}, 'lfs': {'setstripe': {'size': 65536, 'count': 5}}}
S3D-IO 50-50-100-2-2-4-1 208.32 0.03 0.15 153.55 0.15 0.99 0.06 0.02 65536 5 enable enable disable enable 536870912 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'cb_buffer_size': 536870912.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 5.0, 'setstripe-size': 65536.0}, 'iteration': 3, 'iteration_time': 5.0573093527927995, 'status': 'ok'}
 60%|██████    | 3/5 [00:17<00:10,  5.49s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.009931 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'cb_buffer_size': '629145600'}, 'lfs': {'setstripe': {'size': 131072, 'count': 18}}}
S3D-IO 50-50-100-2-2-4-1 122.96 0.03 0.25 35.87 0.15 4.25 0.07 0.12 131072 18 enable disable disable enable 629145600 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'cb_buffer_size': 629145600.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 18.0, 'setstripe-size': 131072.0}, 'iteration': 4, 'iteration_time': 8.493388815782964, 'status': 'ok'}
 80%|████████  | 4/5 [00:25<00:06,  6.40s/it, best loss: 0.003205128205128205]

INFO:hyperopt.tpe:tpe_transform took 0.017770 seconds
INFO:hyperopt.tpe:TPE using 4/4 trials with best loss 0.003205


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'cb_buffer_size': '612368384'}, 'lfs': {'setstripe': {'size': 393216, 'count': 8}}}
S3D-IO 50-50-100-2-2-4-1 168.60 0.03 0.18 178.30 0.15 0.86 0.07 0.01 393216 8 enable enable disable disable 612368384 

312.0                                                                         
{'result': 0.003205128205128205, 'hyperparameters': {'cb_buffer_size': 612368384.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'disable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 8.0, 'setstripe-size': 393216.0}, 'iteration': 5, 'iteration_time': 5.143926255404949, 'status': 'ok'}
100%|██████████| 5/5 [00:30<00:00,  6.04s/it, best loss: 0.003205128205128205]
